## Env Development  
This will contain tests for developing environment.

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import functools
import os
import time

from absl import app
from absl import logging
import sys
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Environment/Simplifications')
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Environment/Variations')
sys.path.insert(1, '/home/jovyan/Masterarbeit/reinforce-one/Agent/DDPG/Test1_Frequent_Returns')

import gin
from six.moves import range
import tensorflow as tf  # pylint: disable=g-explicit-tensorflow-version-import

import matplotlib
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
#plt.ioff() 

from tf_agents.agents.ddpg import actor_rnn_network
from tf_agents.agents.ddpg import critic_rnn_network
from tf_agents.agents.ddpg import ddpg_agent
from tf_agents.drivers import dynamic_episode_driver
from tf_agents.environments import suite_dm_control
from tf_agents.environments import tf_py_environment
from tf_agents.environments import py_environment
from tf_agents.environments import wrappers
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.utils import common

import numpy as np
from tf_agents.environments import utils
from tf_agents.trajectories.time_step import StepType
from tf_agents.trajectories import TimeStep
from tf_agents.policies import scripted_py_policy
from tf_agents.policies import random_py_policy
from tf_agents.policies import policy_saver
from tf_agents.metrics import py_metrics
from tf_agents.drivers import py_driver
from tf_agents.specs import tensor_spec
from tf_agents.networks import sequential

from Env import Env
from Env_No_GT import Env_No_GT
from Env_TSLC import Env_TSLC
from Env_LS import Env_LS
from FR_Env import FREnv
from Env_no_tests import Env_NT
from Env_Simple import Env_S
from Env_S_C import Env_S_C
from Env_S_A import Env_S_A
from Env_Transfers import Env_T
from Env_Cull_Counter import Env_CC
from Env_Complexity import Env_Comp

max_episode_length=1000
num_herds = 2
total_population = 300
weeks_until_testresults = 0.
average_episode_length = 200

In [2]:
#py_env = Env_NT(num_herds = num_herds, total_population = total_population, fix_episode_length=True, average_episode_length = 200)
py_env = Env_TSLC(num_herds = num_herds, 
             total_population = total_population, 
             fix_episode_length=True, 
             weeks_until_testresults = weeks_until_testresults,
             average_episode_length = average_episode_length)

py_env_comp = Env_Comp(num_herds = num_herds,
                 total_population = total_population, 
                 fix_episode_length=True, 
                 weeks_until_testresults = weeks_until_testresults,
                 average_episode_length = average_episode_length)

'''
py_env_t = Env_T(num_herds = num_herds,
                 total_population = total_population, 
                 fix_episode_length=True, 
                 weeks_until_testresults = weeks_until_testresults,
                 average_episode_length = average_episode_length)

py_env_cc = Env_CC(num_herds = num_herds,
                   total_population = total_population, 
                   fix_episode_length=True, 
                   weeks_until_testresults = weeks_until_testresults,
                   average_episode_length = average_episode_length)
'''

'\npy_env_t = Env_T(num_herds = num_herds,\n                 total_population = total_population, \n                 fix_episode_length=True, \n                 weeks_until_testresults = weeks_until_testresults,\n                 average_episode_length = average_episode_length)\n\npy_env_cc = Env_CC(num_herds = num_herds,\n                   total_population = total_population, \n                   fix_episode_length=True, \n                   weeks_until_testresults = weeks_until_testresults,\n                   average_episode_length = average_episode_length)\n'

First, test environment input and output.

In [3]:
if(num_herds==2):
    zero_step = py_env.reset()
    print('Reset: ', zero_step)
    for i in range (0,10):
        step = py_env.step([0.,0.,0.,0.])
        print('Step: ', step)
else:
    print('Not for more than 2 herds.')

Reset:  TimeStep(
{'discount': 1.0,
 'observation': array([0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
 'reward': 0.0,
 'step_type': array(0, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.005, 0.   , 0.   , 0.005, 0.005, 0.   , 0.   , 0.005],
      dtype=float32),
 'reward': array(-0.18666667, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.01, 0.  , 0.  , 0.01, 0.01, 0.  , 0.  , 0.01], dtype=float32),
 'reward': array(-0.21333334, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.015, 0.   , 0.   , 0.015, 0.015, 0.   , 0.   , 0.015],
      dtype=float32),
 'reward': array(-0.25333333, dtype=float32),
 'step_type': array(1, dtype=int32)})
Step:  TimeStep(
{'discount': array(1., dtype=float32),
 'observation': array([0.02, 0.  , 0.  , 0.02, 0.02, 0.  , 0.  , 0.02]

Define two scripted policies:

In [4]:
action_script1 = [(16, [0,0,0,0]), 
                 (1, [0,0,1,1]),
                 (16, [0,0,0,0]), 
                 (1, [0,0,1,1])] * int(1+max_episode_length)

scr_pol_1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script1)

action_script2 = [(17, [0,0,0,0]), 
                 (1, [0,0,1,1]),
                 (17, [0,0,0,0]), 
                 (1, [0,0,1,1])] * int(1+max_episode_length)

scr_pol_2 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script2)

'''action_script1 = [(5, [0,0]), 
                 (1, [1,1]),
                 (5, [0,0]), 
                 (1, [1,1])] * int(1+max_episode_length)

scr_pol_1 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script1)

action_script2 = [(6, [0,0]), 
                 (1, [1,1]),
                 (6, [0,0]), 
                 (1, [1,1])] * int(1+max_episode_length)

scr_pol_2 = scripted_py_policy.ScriptedPyPolicy(
    time_step_spec=py_env.time_step_spec(),
    action_spec=py_env.action_spec(),
    action_script=action_script2)'''

'action_script1 = [(5, [0,0]), \n                 (1, [1,1]),\n                 (5, [0,0]), \n                 (1, [1,1])] * int(1+max_episode_length)\n\nscr_pol_1 = scripted_py_policy.ScriptedPyPolicy(\n    time_step_spec=py_env.time_step_spec(),\n    action_spec=py_env.action_spec(),\n    action_script=action_script1)\n\naction_script2 = [(6, [0,0]), \n                 (1, [1,1]),\n                 (6, [0,0]), \n                 (1, [1,1])] * int(1+max_episode_length)\n\nscr_pol_2 = scripted_py_policy.ScriptedPyPolicy(\n    time_step_spec=py_env.time_step_spec(),\n    action_spec=py_env.action_spec(),\n    action_script=action_script2)'

And create a random policy:

In [5]:
random_policy = random_py_policy.RandomPyPolicy(time_step_spec=py_env.time_step_spec(), 
                                                action_spec=py_env.action_spec())

Now write a function that tests an environment with any policy.  
Outputs average return over a set number of episodes and average steps where the agent culled one or more herds.

In [6]:
def test_env(environment, policy, num_episodes = 50, num_herds = 2):
    if isinstance(environment, py_environment.PyEnvironment):
        total_return = []
        culls = 0 
        if environment.action_spec().shape[0] == num_herds:
            only_culls = True
        else:
            only_culls = False
        for e in range(num_episodes):

            time_step = environment.reset()
            if isinstance(policy, scripted_py_policy.ScriptedPyPolicy):
                policy_state = policy.get_initial_state() # remember where in the script we were
            else:
                #print(policy.get_initial_state(batch_size=train_env.batch_size()))
                policy_state = policy.get_initial_state(batch_size=1) # other policies without memory
            episode_return = 0.0
            i=0
            while not time_step.is_last():
                i+=1
                action_step = policy.action(time_step, policy_state)
                if only_culls:
                    for j in range (0, num_herds):
                        if action_step.action[j] >= 0.5:
                            culls += 1
                else:
                    for j in range (num_herds, num_herds*2):
                        if action_step.action[j] >= 0.5:
                            culls += 1
                policy_state = action_step.state
                time_step = environment.step(action_step.action)

                episode_return += time_step.reward

            total_return.append(episode_return)
        culls /= num_episodes
        average_return = np.average(total_return)
        variance = np.var(total_return)
        standard_deviation = np.std(total_return)
        return culls, average_return, variance, standard_deviation
    else:
        return None

At this point, use the policies for stress testing.

In [7]:
if(num_herds == 2):
    culls, avg_return, var, std = test_env(py_env, scr_pol_1, num_episodes = 200)
    print('average return = {0} culls = {1}'.format(avg_return, culls))

average return = -37.02406674981117 culls = 22.0


In [8]:
if(num_herds == 2):
    culls, avg_return, var, std = test_env(py_env, scr_pol_2, num_episodes = 200)
    print('average return = {0} culls = {1}'.format(avg_return, culls))

average return = -38.39833341243677 culls = 22.0


In [9]:
culls, avg_return, var, std = test_env(py_env, random_policy , num_episodes = 100)
print('average return = {0} culls = {1}'.format(avg_return, culls))

average return = -201.52542569446493 culls = 200.06


Additionally, run a Pseudo-Agent. Currently only works for 2 Herds / Actions with test.

In [10]:
def PseudoAgent(env, num_episodes, cull_threshhold = 0.08):
    assert num_episodes >= 0, "Please enter a positive integer for episode number."
    # Outputs
    total_culls = np.zeros((num_herds,), np.int32)
    total_return = []
    testresults_time = weeks_until_testresults / average_episode_length
    
    for i in range (0, num_episodes):
        time_step = env.reset()
        episode_return = 0.
        
        while not time_step.is_last():
            cull_all = False
            act = np.zeros((num_herds*2,), np.float32)
            for k in range (0, num_herds):
                act[k] = 0.5
            for l in range (0, num_herds):
                if (np.float32(time_step.observation[(l*4)+2]) >= np.float32(cull_threshhold) 
                    and np.float32(time_step.observation[(l*4)]) == np.float32(testresults_time)):
                    cull_all = True
                    break
            if cull_all:
                for j in range (num_herds,num_herds*2):
                    act[j] = 1.
                    total_culls[j-num_herds] += 1
            time_step = env.step(act)
            episode_return += time_step.reward
        total_return.append(episode_return)
        
    culls = np.sum(total_culls) / num_episodes
    average_return = np.average(total_return)
    variance = np.var(total_return)
    standard_deviation = np.std(total_return)
    return culls, average_return, variance, standard_deviation

In [11]:
def PseudoAgent2(env, num_episodes, cull_threshhold = 0.04):
    assert num_episodes >= 0, "Please enter a positive integer for episode number."
    # Outputs
    total_culls = np.zeros((num_herds,), np.int32)
    total_return = []
    testresults_time = weeks_until_testresults / average_episode_length
    
    for i in range (0, num_episodes):
        time_step = env.reset()
        episode_return = 0.
        
        while not time_step.is_last():
            act = np.zeros((num_herds*2,), np.float32)
            for k in range (0, num_herds):
                act[k] = 0.5
            for l in range (0, num_herds):
                if (np.float32(time_step.observation[(l*4)+2]) >= np.float32(cull_threshhold) 
                    and np.float32(time_step.observation[(l*4)]) == np.float32(testresults_time)):
                    act[l+num_herds] = 1.
                    total_culls[l] += 1
            time_step = env.step(act)
            episode_return += time_step.reward
        total_return.append(episode_return)
        
    culls = np.sum(total_culls) / num_episodes
    average_return = np.average(total_return)
    variance = np.var(total_return)
    standard_deviation = np.std(total_return)
    return culls, average_return, variance, standard_deviation

In [12]:
c , ar, var, std = PseudoAgent(py_env, 500, 0.054)  #0.054 ~ 0.056 optimal
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)
print('Variance: ', var)
print('Standard Deviation: ', std)

Average Culls per Episode:  21.612
Average Return:  -32.78680077647604
Variance:  6.688213308973516
Standard Deviation:  2.5861580208822343


In [13]:
c, ar, var, std = PseudoAgent2(py_env, 500, 0.03) #0.03~0.04 optimal
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)
print('Variance: ', var)
print('Standard Deviation: ', std)

Average Culls per Episode:  22.856
Average Return:  -30.80808030181099
Variance:  7.020365043847092
Standard Deviation:  2.649597147463571


In [14]:
c , ar, var, std = PseudoAgent(py_env_comp, 500, 0.1)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)
print('Variance: ', var)
print('Standard Deviation: ', std)

Average Culls per Episode:  17.088
Average Return:  -34.40010728826746
Variance:  6.939945460426102
Standard Deviation:  2.6343776229739926


In [15]:
c , ar, var, std = PseudoAgent2(py_env_comp, 500, 0.05)
print('Average Culls per Episode: ', c)
print('Average Return: ', ar)
print('Variance: ', var)
print('Standard Deviation: ', std)

Average Culls per Episode:  19.612
Average Return:  -30.82821360109933
Variance:  7.143349462933075
Standard Deviation:  2.672704522189663


In [16]:
# Optimize PAs

def optimize_PAs(ct_start = 0.01, ct_end = 0.2, steps = 0.01, environment = py_env, num_episodes = 200):
    ar_list_1 = []
    ar_list_2 = []
    not_end = True
    i = ct_start
    while not_end:
        if i >= (ct_end):
            not_end = False

        c, ar1, var, std = PseudoAgent(environment, num_episodes, i)
        c, ar2, var, std = PseudoAgent2(environment, num_episodes, i)
        ar_list_1.append(ar1)
        ar_list_2.append(ar2)
        i += steps

    ar_max = np.amax(ar_list_1)
    max_index = ar_list_1.index(ar_max)
    ct1_max = ct_start + (steps * max_index)
    ar_max = np.amax(ar_list_2)
    max_index = ar_list_2.index(ar_max)
    ct2_max = ct_start + (steps * max_index)
    
    return ar_list_1, ar_list_2, ct1_max, ct2_max

arl1, arl2, ct1m, ct2m = optimize_PAs(0.01, 0.1, 0.01, py_env, 5000)
print('AR 1 List after first run: ', arl1)
print('CT 1 List after first run: ', ct1m)
print('AR 2 List after first run: ', arl2)
print('CT 2 List after first run: ', ct2m)
ct1_start = ct1m - (3*0.01)
ct1_end = ct1m + (3*0.01)
ct2_start = ct2m - (3*0.01)
ct2_end = ct2m + (3*0.01)

arl1, arl2, ct1m, ct2m = optimize_PAs(ct1_start, ct1_end, 0.001, py_env, 5000)
ar_max = np.amax(arl1)
max_index = arl1.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs after 2nd run: ', arl1[low:high])
print('CT of center Value: ', ct1m)

arl1, arl2, ct1m, ct2m = optimize_PAs(ct2_start, ct2_end, 0.001, py_env, 5000)
ar_max = np.amax(arl2)
max_index = arl2.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs for PA 2: ', arl2[low:high])
print('CT of center Value: ', ct2m)
    
ar_max = np.amax(arl1)
max_index = arl1.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs after third run: ', arl1[low:high])
print('CT of center Value after third run: ', ct1m)

AR 1 List after first run:  [-73.76264784749812, -44.85519610775271, -36.5507237314838, -36.62460640086178, -33.677616885660406, -32.88142478070222, -33.127912705389875, -33.204814040013034, -33.99811531717917, -35.196133936884344, -36.6786458931746]
CT 1 List after first run:  0.060000000000000005
AR 2 List after first run:  [-47.635985893533096, -33.91833353169542, -30.82781630076915, -30.809560304655413, -31.123925597058424, -32.600562858899586, -34.73141880855597, -34.64338413858498, -37.20374677447276, -40.018970756269525, -42.843850736088676]
CT 2 List after first run:  0.04


NameError: name 'ct1_max' is not defined

In [ ]:
ct1_start = ct1m - (3*0.01)
ct1_end = ct1m + (3*0.01)
ct2_start = ct2m - (3*0.01)
ct2_end = ct2m + (3*0.01)

arl1, arl2, ct1m, ct2m = optimize_PAs(ct1_start, ct1_end, 0.001, py_env, 5000)
ar_max = np.amax(arl1)
max_index = arl1.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs after 2nd run: ', arl1[low:high])
print('CT of center Value: ', ct1m)

arl1, arl2, ct1m, ct2m = optimize_PAs(ct2_start, ct2_end, 0.001, py_env, 5000)
ar_max = np.amax(arl2)
max_index = arl2.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs for PA 2: ', arl2[low:high])
print('CT of center Value: ', ct2m)
    
ar_max = np.amax(arl1)
max_index = arl1.index(ar_max)
low = max_index - 5
high = max_index + 5
print('Range of high ARs after third run: ', arl1[low:high])
print('CT of center Value after third run: ', ct1m)

In [ ]:
ar_max = np.amax(arl1)
max_index = arl1.index(ar_max)
low = max_index - 10
high = max_index + 10
print('Range of high ARs after third run: ', arl1[low:high])
print('CT of center Value after third run: ', ct1m)